In [1]:
"""
Generate customers.csv for Data Science Course
==============================================
Uses Faker library for realistic data generation
Optimized for speed with batch operations

Run: pip install faker pandas numpy
"""


'\nGenerate customers.csv for Data Science Course\n==============================================\nUses Faker library for realistic data generation\nOptimized for speed with batch operations\n\nRun: pip install faker pandas numpy\n'

In [2]:
#importing necessary libraries
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import time


In [ ]:
# Initialize
fake = Faker('en_IN')  # Indian locale for realistic Indian data
Faker.seed(42)
np.random.seed(42)

print("="*70)
print("GENERATING CUSTOMERS.CSV")
print("="*70)

# Configuration
NUM_CUSTOMERS = 10000
START_DATE = datetime(2019, 1, 1)
END_DATE = datetime(2024, 12, 31)

start_time = time.time()
print(f"\nGenerating {NUM_CUSTOMERS:,} customers...")

# ============================================================================
# GENERATE CUSTOMER DATA
# ============================================================================

# Generate base data using Faker (batch for speed)
print("  → Generating names and emails...")
customer_data = {
    'Customer_ID': [f'CUST_{i:06d}' for i in range(1, NUM_CUSTOMERS + 1)],
    'First_Name': [fake.first_name() for _ in range(NUM_CUSTOMERS)],
    'Last_Name': [fake.last_name() for _ in range(NUM_CUSTOMERS)],
    'Email': [fake.email() for _ in range(NUM_CUSTOMERS)],
    'Phone': [fake.phone_number() for _ in range(NUM_CUSTOMERS)]
}

print("  → Generating demographics...")

# Age distribution (realistic, slightly right-skewed)
# Using gamma distribution: shape=5, scale=7 gives nice 18-80 range
ages = np.random.gamma(shape=5, scale=7, size=NUM_CUSTOMERS) + 18
ages = np.clip(ages, 18, 80).astype(int)

# Gender distribution
genders = np.random.choice(['Male', 'Female', 'Other'], 
                          size=NUM_CUSTOMERS, 
                          p=[0.48, 0.50, 0.02])

# Location (Indian cities with proper state mapping)
print("  → Generating locations...")
city_state_mapping = {
    'Hyderabad': 'Telangana',
    'Bangalore': 'Karnataka',
    'Mumbai': 'Maharashtra',
    'Delhi': 'Delhi',
    'Chennai': 'Tamil Nadu',
    'Kolkata': 'West Bengal',
    'Pune': 'Maharashtra',
    'Ahmedabad': 'Gujarat',
    'Jaipur': 'Rajasthan',
    'Lucknow': 'Uttar Pradesh',
    'Visakhapatnam': 'Andhra Pradesh',
    'Kochi': 'Kerala',
    'Indore': 'Madhya Pradesh',
    'Chandigarh': 'Punjab',
    'Bhopal': 'Madhya Pradesh'
}

cities = list(city_state_mapping.keys())
city_weights = [0.15, 0.15, 0.12, 0.10, 0.08, 0.07, 0.06, 0.05, 0.04, 0.04, 0.04, 0.03, 0.03, 0.02, 0.02]

customer_cities = np.random.choice(cities, size=NUM_CUSTOMERS, p=city_weights)
customer_states = [city_state_mapping[city] for city in customer_cities]

# Income (correlated with age - older people generally earn more)
print("  → Generating income data...")
# Base income increases with age
base_income = 250000 + (ages - 18) * 12000  
# Add log-normal noise for realistic variation
income_multiplier = np.random.lognormal(mean=0, sigma=0.6, size=NUM_CUSTOMERS)
incomes = (base_income * income_multiplier).astype(int)
incomes = np.clip(incomes, 180000, 6000000)  # Realistic bounds for India

# Customer segment based on income quartiles
income_quartiles = pd.qcut(incomes, q=4, labels=['Budget', 'Standard', 'Premium', 'Luxury'])

# Join date (customer acquisition over time with growth trend)
print("  → Generating join dates...")
days_range = (END_DATE - START_DATE).days

# More customers joined recently (business growth)
# Using beta distribution for realistic growth pattern
join_day_distribution = np.random.beta(a=2, b=5, size=NUM_CUSTOMERS)
random_days = (join_day_distribution * days_range).astype(int)
join_dates = [START_DATE + timedelta(days=int(d)) for d in random_days]

# Account status (most active, some inactive)
account_status = np.random.choice(['Active', 'Inactive', 'Suspended'], 
                                  size=NUM_CUSTOMERS,
                                  p=[0.85, 0.13, 0.02])

# Preferred language
languages = np.random.choice(['English', 'Hindi', 'Telugu', 'Tamil', 'Bengali'], 
                            size=NUM_CUSTOMERS,
                            p=[0.40, 0.30, 0.12, 0.10, 0.08])

# Loyalty points (based on tenure)
days_as_customer = [(END_DATE - jd).days for jd in join_dates]
loyalty_points = (np.array(days_as_customer) * np.random.uniform(0.5, 2.0, NUM_CUSTOMERS)).astype(int)
loyalty_points = np.clip(loyalty_points, 0, 50000)

# ============================================================================
# CREATE DATAFRAME
# ============================================================================

print("  → Assembling dataframe...")

customers_df = pd.DataFrame({
    'Customer_ID': customer_data['Customer_ID'],
    'First_Name': customer_data['First_Name'],
    'Last_Name': customer_data['Last_Name'],
    'Age': ages,
    'Gender': genders,
    'City': customer_cities,
    'State': customer_states,
    'Income': incomes,
    'Email': customer_data['Email'],
    'Phone': customer_data['Phone'],
    'Join_Date': join_dates,
    'Customer_Segment': income_quartiles,
    'Account_Status': account_status,
    'Preferred_Language': languages,
    'Loyalty_Points': loyalty_points
})

# ============================================================================
# ADD REALISTIC DATA QUALITY ISSUES
# ============================================================================

print("  → Adding realistic missing values...")

# Missing values (2-5% is realistic)
missing_indices_income = np.random.choice(NUM_CUSTOMERS, size=int(NUM_CUSTOMERS * 0.03), replace=False)
customers_df.loc[missing_indices_income, 'Income'] = np.nan

missing_indices_phone = np.random.choice(NUM_CUSTOMERS, size=int(NUM_CUSTOMERS * 0.02), replace=False)
customers_df.loc[missing_indices_phone, 'Phone'] = np.nan

missing_indices_language = np.random.choice(NUM_CUSTOMERS, size=int(NUM_CUSTOMERS * 0.01), replace=False)
customers_df.loc[missing_indices_language, 'Preferred_Language'] = np.nan

# ============================================================================
# SAVE TO CSV
# ============================================================================

print("  → Saving to customers.csv...")
customers_df.to_csv(r'./customers.csv', index=False)

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================

elapsed_time = time.time() - start_time

print("\n" + "="*70)
print("✅ CUSTOMERS.CSV GENERATED SUCCESSFULLY!")
print("="*70)
print(f"\nTime taken: {elapsed_time:.2f} seconds")
print(f"Total customers: {len(customers_df):,}")
print(f"File size: {customers_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

print("\n📊 DATA SUMMARY:")
print("-"*70)
print(f"Age range: {customers_df['Age'].min()} - {customers_df['Age'].max()} years")
print(f"Average age: {customers_df['Age'].mean():.1f} years")
print(f"Income range: ₹{customers_df['Income'].min():,.0f} - ₹{customers_df['Income'].max():,.0f}")
print(f"Average income: ₹{customers_df['Income'].mean():,.0f}")

print("\n📍 TOP 5 CITIES:")
print(customers_df['City'].value_counts().head())

print("\n👥 GENDER DISTRIBUTION:")
print(customers_df['Gender'].value_counts())

print("\n💎 CUSTOMER SEGMENTS:")
print(customers_df['Customer_Segment'].value_counts())

print("\n⚠️ MISSING VALUES:")
print(customers_df.isnull().sum()[customers_df.isnull().sum() > 0])

print("\n📋 SAMPLE DATA (first 5 rows):")
print(customers_df.head())
print("="*70)

GENERATING CUSTOMERS.CSV

Generating 10,000 customers...
  → Generating names and emails...
  → Generating demographics...
  → Generating locations...
  → Generating income data...
  → Generating join dates...
  → Assembling dataframe...
  → Adding realistic missing values...
  → Saving to customers.csv...

✅ CUSTOMERS.CSV GENERATED SUCCESSFULLY!

Time taken: 0.35 seconds
Total customers: 10,000
File size: 5.83 MB

📊 DATA SUMMARY:
----------------------------------------------------------------------
Age range: 20 - 80 years
Average age: 52.1 years
Income range: ₹180,000 - ₹6,000,000
Average income: ₹788,682

📍 TOP 5 CITIES:
City
Hyderabad    1509
Bangalore    1504
Mumbai       1194
Delhi        1011
Chennai       817
Name: count, dtype: int64

👥 GENDER DISTRIBUTION:
Gender
Female    4958
Male      4853
Other      189
Name: count, dtype: int64

💎 CUSTOMER SEGMENTS:
Customer_Segment
Budget      2500
Standard    2500
Premium     2500
Luxury      2500
Name: count, dtype: int64

⚠️ MISSING